# <b>Deep Learning:</b> Risiko! Army detector

---
A.A. 2022/23 (6 CFU) - Dr. Daniel Fusaro Ph.D. Student

---

# Yolov7 test


With this Jupyter Notebook you can:
 - test the accuracy of a pre-trained Yolov7 model on your custom dataset
 - get the performance metrics (that you need to include in your project report)
 - visualize the results

---

Risiko! game official Wikipedia page -
[https://en.wikipedia.org/wiki/RisiKo!](https://en.wikipedia.org/wiki/RisiKo!)

---

### To run this Jupyter Notebook you will need

#### run the following commands:
```
pip3 install torch torchvision
pip3 install seaborn
git clone https://github.com/WongKinYiu/yolov7
 ```
#### follow these steps:
 - the weights of the pretrained model (see the Google Drive folder of the project)
 - the dataset produced by you (using Risiko! Synthetic Dataset Creator.ipynb)
 - the dataset of real images that you find in the Google Drive folder of the project
 - modify the coco_risiko.yaml config file: it must contain, in the "test" value, the path to the test.txt file containing all the test images path
 - produce the aforementioned test.txt file, like the one you find in the Google Drive folder (aka test_example.txt)


In [1]:
!pip install torch torchvision
!pip install seaborn
!git clone https://github.com/WongKinYiu/yolov7
!pip install PyQt5

Cloning into 'yolov7'...
remote: Enumerating objects: 1191, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1191 (delta 2), reused 6 (delta 2), pack-reused 1185
Receiving objects: 100% (1191/1191), 74.22 MiB | 30.52 MiB/s, done.
Resolving deltas: 100% (513/513), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 11.5 MB/s eta 0:00:00


In [2]:
import argparse
import json
import os
from pathlib import Path
from threading import Thread

import numpy as np
import torch                    # if you don't have it, just "pip3 install torch torchvision"
import yaml
from tqdm import tqdm

import matplotlib
from matplotlib import pyplot as plt
import cv2
import yaml

import sys
sys.path.append('yolov7')

# probably, you also must do "pip3 install seaborn"

from models.experimental import attempt_load
from utils.datasets import create_dataloader
from utils.general import coco80_to_coco91_class, check_dataset, check_file, check_img_size, check_requirements, \
    box_iou, non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, set_logging, increment_path, colorstr
from utils.metrics import ap_per_class, ConfusionMatrix
from utils.plots import plot_images, output_to_target, plot_study_txt
from utils.torch_utils import select_device, time_synchronized, TracedModel

## The "test" function

In [3]:
def test(data,
         weights=None,
         batch_size=32,
         imgsz=640,
         conf_thres=0.001,
         iou_thres=0.6,      # for NMS
         save_json=False,
         single_cls=False,
         augment=False,
         verbose=False,
         model=None,
         dataloader=None,
         save_dir=Path(''),  # for saving images
         save_txt=False,     # for auto-labelling
         save_hybrid=False,  # for hybrid auto-labelling
         save_conf=False,    # save auto-label confidences
         plots=True,
         wandb_logger=None,
         compute_loss=None,
         half_precision=True,
         trace=False,
         is_coco=False,
         v5_metric=False):

    set_logging()
    device = select_device(opt.device, batch_size=batch_size)

    # Directories
    save_dir = Path(increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok))  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    gs = max(int(model.stride.max()), 32)  # grid size (max stride)
    imgsz = check_img_size(imgsz, s=gs)  # check img_size

    if trace:
        model = TracedModel(model, device, imgsz)

    # Half
    half = device.type != 'cpu' and half_precision  # half precision only supported on CUDA
    if half:
        model.half()

    # Configure
    model.eval()
    if isinstance(data, str):
        is_coco = data.endswith('coco.yaml')
        with open(data) as f:
            data = yaml.load(f, Loader=yaml.SafeLoader)
    check_dataset(data)  # check
    nc = 1 if single_cls else int(data['nc'])  # number of classes
    iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
    niou = iouv.numel()

    # Logging
    log_imgs = 0
    if wandb_logger and wandb_logger.wandb:
        log_imgs = min(wandb_logger.log_imgs, 100)
    # Dataloader
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    task = opt.task if opt.task in ('train', 'val', 'test') else 'val'  # path to train/val/test images
    dataloader = create_dataloader(data[task], imgsz, batch_size, gs, opt, pad=0.5, rect=True,
                                   prefix=colorstr(f'{task}: '))[0]

    if v5_metric:
        print("Testing with YOLOv5 AP metric...")

    seen = 0
    confusion_matrix = ConfusionMatrix(nc=nc)
    names = {k: v for k, v in enumerate(model.names if hasattr(model, 'names') else model.module.names)}
    coco91class = coco80_to_coco91_class()
    s = ('%20s' + '%12s' * 6) % ('Class', 'Images', 'Labels', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')
    p, r, f1, mp, mr, map50, map, t0, t1 = 0., 0., 0., 0., 0., 0., 0., 0., 0.
    loss = torch.zeros(3, device=device)
    jdict, stats, ap, ap_class, wandb_images = [], [], [], [], []
    for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader, desc=s)):
        img = img.to(device, non_blocking=True)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        targets = targets.to(device)
        nb, _, height, width = img.shape  # batch size, channels, height, width

        with torch.no_grad():
            # Run model
            t = time_synchronized()
            out, train_out = model(img, augment=augment)  # inference and training outputs
            t0 += time_synchronized() - t

            # Compute loss
            if compute_loss:
                loss += compute_loss([x.float() for x in train_out], targets)[1][:3]  # box, obj, cls

            # Run NMS
            targets[:, 2:] *= torch.Tensor([width, height, width, height]).to(device)  # to pixels
            lb = [targets[targets[:, 0] == i, 1:] for i in range(nb)] if save_hybrid else []  # for autolabelling

            t = time_synchronized()
            out = non_max_suppression(out, conf_thres=conf_thres, iou_thres=iou_thres, labels=lb, multi_label=True)
            t1 += time_synchronized() - t

        # Statistics per image
        for si, pred in enumerate(out):
            labels = targets[targets[:, 0] == si, 1:]
            nl = len(labels)
            tcls = labels[:, 0].tolist() if nl else []  # target class
            path = Path(paths[si])
            seen += 1

            if len(pred) == 0:
                if nl:
                    stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
                continue

            # Predictions
            predn = pred.clone()
            scale_coords(img[si].shape[1:], predn[:, :4], shapes[si][0], shapes[si][1])  # native-space pred

            if opt.print_bbox:
                for line in predn:
                    x, y, w, h, conf, cl = line.detach().cpu().float()
                    print(f"{int(x):<6.0f} {int(y):<6.0f} {int(w):<6.0f} {int(h):<6.0f} {conf:<6.4} {int(cl):<2d}")

            opt.labels.append([predn, paths[si]])

            # Append to text file
            if save_txt:
                gn = torch.tensor(shapes[si][0])[[1, 0, 1, 0]]  # normalization gain whwh
                for *xyxy, conf, cls in predn.tolist():
                    xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                    line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                    with open(save_dir / 'labels' / (path.stem + '.txt'), 'a') as f:
                        f.write(('%g ' * len(line)).rstrip() % line + '\n')

            # W&B logging - Media Panel Plots
            if len(wandb_images) < log_imgs and wandb_logger.current_epoch > 0:  # Check for test operation
                if wandb_logger.current_epoch % wandb_logger.bbox_interval == 0:
                    box_data = [{"position": {"minX": xyxy[0], "minY": xyxy[1], "maxX": xyxy[2], "maxY": xyxy[3]},
                                 "class_id": int(cls),
                                 "box_caption": "%s %.3f" % (names[cls], conf),
                                 "scores": {"class_score": conf},
                                 "domain": "pixel"} for *xyxy, conf, cls in pred.tolist()]
                    boxes = {"predictions": {"box_data": box_data, "class_labels": names}}  # inference-space
                    wandb_images.append(wandb_logger.wandb.Image(img[si], boxes=boxes, caption=path.name))
            wandb_logger.log_training_progress(predn, path, names) if wandb_logger and wandb_logger.wandb_run else None

            # Append to pycocotools JSON dictionary
            if save_json:
                # [{"image_id": 42, "category_id": 18, "bbox": [258.15, 41.29, 348.26, 243.78], "score": 0.236}, ...
                image_id = int(path.stem) if path.stem.isnumeric() else path.stem
                box = xyxy2xywh(predn[:, :4])  # xywh
                box[:, :2] -= box[:, 2:] / 2  # xy center to top-left corner
                for p, b in zip(pred.tolist(), box.tolist()):
                    jdict.append({'image_id': image_id,
                                  'category_id': coco91class[int(p[5])] if is_coco else int(p[5]),
                                  'bbox': [round(x, 3) for x in b],
                                  'score': round(p[4], 5)})

            # Assign all predictions as incorrect
            correct = torch.zeros(pred.shape[0], niou, dtype=torch.bool, device=device)
            if nl:
                detected = []  # target indices
                tcls_tensor = labels[:, 0]

                # target boxes
                tbox = xywh2xyxy(labels[:, 1:5])
                scale_coords(img[si].shape[1:], tbox, shapes[si][0], shapes[si][1])  # native-space labels
                if plots:
                    confusion_matrix.process_batch(predn, torch.cat((labels[:, 0:1], tbox), 1))

                # Per target class
                for cls in torch.unique(tcls_tensor):
                    ti = (cls == tcls_tensor).nonzero(as_tuple=False).view(-1)  # prediction indices
                    pi = (cls == pred[:, 5]).nonzero(as_tuple=False).view(-1)  # target indices

                    # Search for detections
                    if pi.shape[0]:
                        # Prediction to target ious
                        ious, i = box_iou(predn[pi, :4], tbox[ti]).max(1)  # best ious, indices

                        # Append detections
                        detected_set = set()
                        for j in (ious > iouv[0]).nonzero(as_tuple=False):
                            d = ti[i[j]]  # detected target
                            if d.item() not in detected_set:
                                detected_set.add(d.item())
                                detected.append(d)
                                correct[pi[j]] = ious[j] > iouv  # iou_thres is 1xn
                                if len(detected) == nl:  # all targets already located in image
                                    break
            # Append statistics (correct, conf, pcls, tcls)
            stats.append((correct.cpu(), pred[:, 4].cpu(), pred[:, 5].cpu(), tcls))

        # Plot images
        if plots and batch_i < 3:
            f = save_dir / f'test_batch{batch_i}_labels.jpg'  # labels
            Thread(target=plot_images, args=(img, targets, paths, f, names), daemon=True).start()
            f = save_dir / f'test_batch{batch_i}_pred.jpg'  # predictions
            Thread(target=plot_images, args=(img, output_to_target(out), paths, f, names), daemon=True).start()

    # Compute statistics
    stats = [np.concatenate(x, 0) for x in zip(*stats)]  # to numpy
    if len(stats) and stats[0].any():
        p, r, ap, f1, ap_class = ap_per_class(*stats, plot=plots, v5_metric=v5_metric, save_dir=save_dir, names=names)
        ap50, ap = ap[:, 0], ap.mean(1)  # AP@0.5, AP@0.5:0.95
        mp, mr, map50, map = p.mean(), r.mean(), ap50.mean(), ap.mean()
        nt = np.bincount(stats[3].astype(np.int64), minlength=nc)  # number of targets per class
    else:
        nt = torch.zeros(1)

    # Print results
    pf = '%20s' + '%12i' * 2 + '%12.3g' * 4  # print format
    print(pf % ('all', seen, nt.sum(), mp, mr, map50, map))

    # Print results per class
    if (verbose or nc < 50) and nc > 1 and len(stats):
        for i, c in enumerate(ap_class):
            print(pf % (names[c], seen, nt[c], p[i], r[i], ap50[i], ap[i]))

    # Print speeds
    t = tuple(x / seen * 1E3 for x in (t0, t1, t0 + t1)) + (imgsz, imgsz, batch_size)  # tuple
    print('Speed: %.1f/%.1f/%.1f ms inference/NMS/total per %gx%g image at batch-size %g' % t)

    # Plots
    if plots:
        confusion_matrix.plot(save_dir=save_dir, names=list(names.values()))
        if wandb_logger and wandb_logger.wandb:
            val_batches = [wandb_logger.wandb.Image(str(f), caption=f.name) for f in sorted(save_dir.glob('test*.jpg'))]
            wandb_logger.log({"Validation": val_batches})
    if wandb_images:
        wandb_logger.log({"Bounding Box Debugger/Images": wandb_images})

    # Save JSON
    if save_json and len(jdict):
        w = Path(weights[0] if isinstance(weights, list) else weights).stem if weights is not None else ''  # weights
        anno_json = './coco/annotations/instances_val2017.json'  # annotations json
        pred_json = str(save_dir / f"{w}_predictions.json")  # predictions json
        print('\nEvaluating pycocotools mAP... saving %s...' % pred_json)
        with open(pred_json, 'w') as f:
            json.dump(jdict, f)

        try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoEvalDemo.ipynb
            from pycocotools.coco import COCO
            from pycocotools.cocoeval import COCOeval

            anno = COCO(anno_json)  # init annotations api
            pred = anno.loadRes(pred_json)  # init predictions api
            eval = COCOeval(anno, pred, 'bbox')
            if is_coco:
                eval.params.imgIds = [int(Path(x).stem) for x in dataloader.dataset.img_files]  # image IDs to evaluate
            eval.evaluate()
            eval.accumulate()
            eval.summarize()
            map, map50 = eval.stats[:2]  # update results (mAP@0.5:0.95, mAP@0.5)
        except Exception as e:
            print(f'pycocotools unable to run: {e}')

    # Return results
    s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
    print(f"Results saved to {save_dir}{s}")

    maps = np.zeros(nc) + map
    for i, c in enumerate(ap_class):
        maps[c] = ap[i]
    return (mp, mr, map50, map, *(loss.cpu() / len(dataloader)).tolist()), maps, t

# Main

In [4]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# access the folder containing the files required to run the project
%cd /content/drive/My Drive/Colab environments/Risiko! DL/
# check that we are in the desired folder and that all required files are present
%ls

Mounted at /content/drive
/content/drive/My Drive/Colab environments/Risiko! DL
 3D_models/
 backgrounds/
 coco_risiko.yaml
 custom_complete_yolo.yaml
 custom_yolo.yaml
 datasets/
 pre_trained_weights/
 README.md
 real_images/
'Risiko!_Synthetic_Dataset_Creator.ipynb'
'Risiko! Test.ipynb'
 Split_train_test_val.ipynb
 synthetic_dataset/
 synthetic_images/
 Tanks_flags_detection_training.ipynb
 Test_detection.ipynb
 test_example.txt
 test.txt
 training_results/
 yolov5/


In [5]:
class Opt:
    data        = "coco_risiko.yaml"
    weights     =  "pre_trained_weights/pre_trained_yolo_weights_v00.pt" ##### PUT HERE the path to the weights of yolo (e.h. pre_trained_weights/pre_trained_yolo_weights_v00.pt)

    batch_size  = 1                           # size of each image batch
    img_size    = 640                         # inference size (pixels)
    conf_thres  = 0.2                         # object confidence threshold
    iou_thres   = 0.65                        # IOU threshold for NMS
    save_json   = False                       # save a cocoapi-compatible JSON results file
    single_cls  = False
    augment     = False
    verbose     = False
    save_txt    = False                        # save results to *.txt
    save_hybrid = False                        # save label+prediction hybrid results to *.txt
    save_conf   = False                        # save confidences in --save-txt labels
    project     = "runs/test"
    no_trace    = False                        # don`t trace model
    v5_metric   = False                        # assume maximum recall as 1.0 in AP calculation
    exist_ok    = False                        # existing project/name ok, do not increment
    device      = "0"                         # cuda device, i.e. 0 or 0,1,2,3 or cpu
    task        = "test"                      #
    name        = "yolov7_risiko_test"        # save to project/name

    print_bbox  = False

    labels = []

In [6]:
opt = Opt()

opt.data = check_file(opt.data)

test(opt.data,
     opt.weights,
     opt.batch_size,
     opt.img_size,
     opt.conf_thres,
     opt.iou_thres,
     opt.save_json,
     opt.single_cls,
     opt.augment,
     opt.verbose,
     save_txt = opt.save_txt | opt.save_hybrid,
     save_hybrid = opt.save_hybrid,
     save_conf = opt.save_conf,
     trace = not opt.no_trace,
     v5_metric = opt.v5_metric
     )

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
test: Scanning '/content/drive/My Drive/Colab environments/Risiko! DL/test' images and labels... 6 found, 0 missing, 0 empty, 0 corrupted:   1%|          | 6/1045 [00:04<12:05,  1.43it/s]

test: WARNING: Ignoring corrupted image and/or label real_images/images/000006.jpg: [Errno 2] No such file or directory: 'real_images/images/000006.jpg'


test: Scanning '/content/drive/My Drive/Colab environments/Risiko! DL/test' images and labels... 10 found, 0 missing, 0 empty, 1 corrupted:   1%|          | 11/1045 [00:07<10:14,  1.68it/s]

test: WARNING: Ignoring corrupted image and/or label real_images/images/000011.jpg: [Errno 2] No such file or directory: 'real_images/images/000011.jpg'


...

test: Scanning '/content/drive/My Drive/Colab environments/Risiko! DL/test' images and labels... 191 found, 0 missing, 0 empty, 854 corrupted: 100%|██████████| 1045/1045 [01:45<00:00,  9.89it/s]


test: WARNING: Ignoring corrupted image and/or label synthetic_dataset/images/000771.jpg: [Errno 2] No such file or directory: 'synthetic_dataset/images/000771.jpg'
test: WARNING: Ignoring corrupted image and/or label synthetic_dataset/images/000772.jpg: [Errno 2] No such file or directory: 'synthetic_dataset/images/000772.jpg'
test: WARNING: Ignoring corrupted image and/or label synthetic_dataset/images/000773.jpg: [Errno 2] No such file or directory: 'synthetic_dataset/images/000773.jpg'
test: WARNING: Ignoring corrupted image and/or label synthetic_dataset/images/000774.jpg: [Errno 2] No such file or directory: 'synthetic_dataset/images/000774.jpg'
test: WARNING: Ignoring corrupted image and/or label synthetic_dataset/images/000775.jpg: [Errno 2] No such file or directory: 'synthetic_dataset/images/000775.jpg'
test: WARNING: Ignoring corrupted image and/or label synthetic_dataset/images/000776.jpg: [Errno 2] No such file or directory: 'synthetic_dataset/images/000776.jpg'
test: WARN

               Class      Images      Labels           P           R      mAP@.5  mAP@.5:.95: 100%|██████████| 191/191 [00:40<00:00,  4.66it/s]


                 all         191       11044       0.904       0.678       0.685       0.531
           blue_army         191        1644       0.902       0.617       0.627       0.447
            red_army         191        1524       0.925       0.619       0.633       0.454
         yellow_army         191        1548       0.944       0.611       0.629       0.461
         purple_army         191        1416       0.908       0.724       0.735       0.532
          black_army         191        1512       0.881       0.566       0.577       0.415
          green_army         191        1330       0.896       0.662       0.654       0.491
           blue_flag         191         408       0.938       0.684       0.693       0.542
            red_flag         191         348       0.906       0.704       0.716       0.609
         yellow_flag         191         334        0.94       0.698       0.712       0.588
         purple_flag         191         344       0.901       0.741  

((0.904393586495232,
  0.6780629518311913,
  0.6848005872300944,
  0.5310153484628,
  0.0,
  0.0,
  0.0),
 array([    0.44698,      0.4539,     0.46062,      0.5325,     0.41497,     0.49093,     0.54226,      0.6092,     0.58824,     0.60891,     0.61947,     0.60421]),
 (28.58221343674585, 1.6621270104852646, 30.24434044723111, 640, 640, 1))

In [8]:
# We don't run this cell because we obtain some error using matplotlib.use('qtagg')
# We tried to install library like PyQt5 to fix this problem but the error still remain
# Another try was to remove the instruction matplotlib.use('qtagg') or changing it into matplotlib.use('agg') but in both cases the output was only a list
# of filename that is not interesting or relevant to the goal of this cell
# We have also asked the tutor assistant how to solve the issue, but he was not able to do that.

# matplotlib.use('qtagg')



#for labels, image_path in opt.labels:
#    img = cv2.imread(image_path)
#    print(image_path)
#    for line in labels:
#        x, y, w, h, conf, cl = line.detach().cpu().float()
#        cv2.rectangle(img, (int(x), int(y)), (int(w), int(h)), (36,255,12, 1) if cl < 6 else (255,12,10, 1), 2)
#    plt.imshow(img)
#    plt.show()
